In [2]:
# import os
# import shutil
# import random

# def partition_images(base_dir, train_dir, test_dir, train_ratio):
#     os.makedirs(train_dir, exist_ok=True)
#     os.makedirs(test_dir, exist_ok=True)

#     all_images = os.listdir(base_dir)
#     random.shuffle(all_images)

#     train_count = int(len(all_images) * train_ratio)
#     train_images = all_images[:train_count]
#     test_images = all_images[train_count:]

#     for image in train_images:
#         shutil.copyfile(os.path.join(base_dir, image), os.path.join(train_dir, image))

#     for image in test_images:
#         shutil.copyfile(os.path.join(base_dir, image), os.path.join(test_dir, image))

# partition_images('images', 'train', 'test', 0.8)


In [3]:
# #class CNN(nn.Module):
#     def __init__(self):
#         super(CNN, self).__init__()
#         self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
#         self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
#         self.relu = nn.ReLU()
#         self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
#         self.dropout = nn.Dropout(0.5)  # Added dropout layer
#         self.fc1 = nn.Linear(64 * 73 * 73, 256)  # Adjusted input size
#         self.fc2 = nn.Linear(256, 2)  # predicting two variables (alfa x and betax)

#     def forward(self, x):
#         x = self.conv1(x)
#         x = self.relu(x)
#         x = self.maxpool(x)
#         x = self.dropout(x)  # Apply dropout after maxpool
#         x = self.conv2(x)
#         x = self.relu(x)
#         x = self.maxpool(x)
#         x = self.dropout(x)  # Apply dropout after maxpool
#         x = x.view(x.size(0), -1)
#         x = self.fc1(x)
#         x = self.relu(x)
#         x = self.fc2(x)
#         return x

    # def __init__(self):
    #     super(CNN, self).__init__()
    #     self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
    #     self.relu = nn.ReLU()
    #     self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
    #     self.fc1 = nn.Linear(16 * 147 * 147, 256)
    #     self.fc2 = nn.Linear(256, 2) # predicting two variables (alfa x and betax)

    # def forward(self, x):
    #     x = self.conv1(x)
    #     x = self.relu(x)
    #     x = self.maxpool(x)
    #     x = x.view(x.size(0), -1)
    #     x = self.fc1(x)
    #     x = self.relu(x)
    #     x = self.fc2(x)
    #     return x



In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import os
import re
import cv2
import numpy as np
from torchvision import transforms
import random

# ensure determinism
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(16 * 147 * 147, 256)
        self.fc2 = nn.Linear(256, 2) # predicting two variables (alfa x and betax)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

class CustomDataset(Dataset):
    def __init__(self, image_files):
        self.image_files = image_files
        self.pattern = r"epsnx([\d.-]+)_alfax([\d.-]+)_betax([\d.-]+)_epsny([\d.-]+)_alfay([\d.-]+)_betay([\d.-]+)_epsnz([\d.-]+)_alfaz([\d.-]+)_betaz([\d.-]+)\.png"
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, index):
        image_path = self.image_files[index]
        image_name = os.path.basename(image_path)
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Resize the image to 295x295
        image = cv2.resize(image, (295, 295))
        
        image = self.transform(image)
        matches = re.search(self.pattern, image_name)
        variables = [float(matches.group(i)) for i in range(1, 10) if matches.group(i)]
        alfa_x = variables[1] # alfax value
        beta_x = variables[2] # betax value
        return image, torch.tensor([alfa_x, beta_x]) # returns alfa x and betax as the labels

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# checkpoint = torch.load(filename)
# model.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
# num_epochs = checkpoint['epoch']
# loss = checkpoint['loss']

In [5]:
from sklearn.model_selection import ParameterGrid
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


# Load the dataset
image_dir = "./hostinger-images-80bin/2-param/"
image_files = [os.path.join(image_dir, name) for name in sorted(os.listdir(image_dir))]
total_size = len(image_files)
train_prop = 0.85
test_prop = 0.15
train_size = int(train_prop * total_size)
test_size = total_size - train_size

# Split the dataset, removed validation network
train_files = image_files[:train_size]
test_files = image_files[train_size:]

train_dataset = CustomDataset(train_files)
test_dataset = CustomDataset(test_files)

# define your parameter grid
param_grid = {
    'batch_size': [4],
    'learning_rate': [0.001],
    'num_epochs': [500]
}

grid = ParameterGrid(param_grid)

for params in grid:
    batch_size = params['batch_size']
    learning_rate = params['learning_rate']
    num_epochs = params['num_epochs']

    filename = f"cnn_model3_bs{batch_size}_lr{learning_rate}_e{num_epochs}.pth"

    # If the model file already exists, skip this iteration
    if os.path.isfile(filename):
        print(f"Model file {filename} already exists. Skipping...")
        continue
    
    batch_size = params['batch_size']
    learning_rate = params['learning_rate']
    num_epochs = params['num_epochs']

    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    model = CNN().to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Train the model
    for epoch in range(num_epochs):
        # Training
        model.train()  # set the model to training mode
        for images, labels in train_dataloader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f"Epoch [{epoch + 1}/{num_epochs}], Train Loss: {loss.item():.4f}")

    # Test
    model.eval()  # set the model to evaluation mode
    with torch.no_grad():
        total_loss = 0
        for images, labels in test_dataloader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
        avg_loss = total_loss / len(test_dataloader)
    print(f"Test Loss: {avg_loss:.4f}")

    filename = f"cnn_model3_bs{batch_size}_lr{learning_rate}_e{num_epochs}.pth"
    torch.save(model.state_dict(), filename)


cuda
Epoch [1/400], Train Loss: 8656.7930
Epoch [2/400], Train Loss: 12374.5098
Epoch [3/400], Train Loss: 1620.7599
Epoch [4/400], Train Loss: 12022.5010
Epoch [5/400], Train Loss: 4509.9951
Epoch [6/400], Train Loss: 2474.6177
Epoch [7/400], Train Loss: 925.8811
Epoch [8/400], Train Loss: 623.4400
Epoch [9/400], Train Loss: 1542.4626
Epoch [10/400], Train Loss: 1248.3260
Epoch [11/400], Train Loss: 3291.0969
Epoch [12/400], Train Loss: 5722.4170
Epoch [13/400], Train Loss: 139.1386
Epoch [14/400], Train Loss: 420.1382
Epoch [15/400], Train Loss: 381.6141
Epoch [16/400], Train Loss: 1792.1631
Epoch [17/400], Train Loss: 255.5192
Epoch [18/400], Train Loss: 470.4072
Epoch [19/400], Train Loss: 3114.0107
Epoch [20/400], Train Loss: 1279.9836
Epoch [21/400], Train Loss: 635.3932
Epoch [22/400], Train Loss: 1061.4446
Epoch [23/400], Train Loss: 1606.2670
Epoch [24/400], Train Loss: 37.4555
Epoch [25/400], Train Loss: 191.5238
Epoch [26/400], Train Loss: 154.2763
Epoch [27/400], Train Loss

In [11]:
# Model evaluation function
def evaluate_model(model, test_dataset):
    model.eval()
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

    total_mae = torch.zeros(2, device=device)
    total_mape = torch.zeros(2, device=device)
    total_smape = torch.zeros(2, device=device)
    total_mse = torch.zeros(2, device=device)
    total_count = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)

            absolute_error = torch.abs(outputs - labels)
            total_mae += absolute_error.sum(dim=0)

            non_zero_mask = torch.abs(labels) > 1e-8
            percentage_error = (absolute_error / torch.abs(labels)) * 100
            total_mape += (percentage_error * non_zero_mask).sum(dim=0)

            smape = 200.0 * torch.abs(outputs - labels) / (torch.abs(outputs) + torch.abs(labels) + torch.finfo(torch.float32).eps)
            total_smape += smape.sum(dim=0)
            
            mse = (outputs - labels) ** 2
            total_mse += mse.sum(dim=0)

            total_count += labels.size(0)

    mae = total_mae / total_count
    mape = total_mape / total_count
    smape = total_smape / total_count
    rmse = torch.sqrt(total_mse / total_count)

    return mae.cpu().numpy(), mape.cpu().numpy(), smape.cpu().numpy(), rmse.cpu().numpy()


In [12]:
import glob

# Path to your model directory
model_dir = './'

# Sort models based on performance
model_performance = []

# For each model file in the directory
for model_file in glob.glob(model_dir + '/*.pth'):
    # Load model state
    model = CNN()
    model.load_state_dict(torch.load(model_file))
    model.to(device)

    # Evaluate model
    mae, mape, smape, rmse = evaluate_model(model, test_dataset)  # Pass test_dataset instead of "test"

    # Calculate some form of aggregate score. Here, I use the mean of all metrics.
    # Change this if you want a different form of ranking.
    aggregate_score = np.mean([np.mean(mae), np.mean(mape), np.mean(smape), np.mean(rmse)])

    # Save model's performance
    model_performance.append((model_file, mae, mape, smape, rmse, aggregate_score))

# Sort models based on the aggregate score
model_performance.sort(key=lambda x: x[5])

# Print model performance in order
for model_info in model_performance:
    model_file, mae, mape, smape, rmse, aggregate_score = model_info
    print(f"Model: {model_file}, Aggregate Score: {aggregate_score:.4f}")
    for i in range(2):
        print(f"Metrics for prediction {i+1}:")
        print(f"Mean Absolute Error: {mae[i]:.4f}")
        print(f"Mean Absolute Percentage Error: {mape[i]:.2f}%")
        print(f"Symmetric Mean Absolute Percentage Error: {smape[i]:.2f}%")
        print(f"Root Mean Square Error: {rmse[i]:.4f}")
    print()

# Open the text document in write mode
with open('model_performance_model_test.txt', 'w') as f:
    # Print model performance in order
    for model_info in model_performance:
        model_file, mae, mape, smape, rmse, aggregate_score = model_info
        f.write(f"Model: {model_file}, Aggregate Score: {aggregate_score:.4f}\n")
        for i in range(2):
            f.write(f"Metrics for prediction {i+1}:\n")
            f.write(f"Mean Absolute Error: {mae[i]:.4f}\n")
            f.write(f"Mean Absolute Percentage Error: {mape[i]:.2f}%\n")
            f.write(f"Symmetric Mean Absolute Percentage Error: {smape[i]:.2f}%\n")
            f.write(f"Root Mean Square Error: {rmse[i]:.4f}\n")
        f.write("\n")



Model: .\cnn_model3_bs4_lr0.001_e400.pth, Aggregate Score: 18.8496
Metrics for prediction 1:
Mean Absolute Error: 0.5984
Mean Absolute Percentage Error: 13.66%
Symmetric Mean Absolute Percentage Error: 14.91%
Root Mean Square Error: 0.7416
Metrics for prediction 2:
Mean Absolute Error: 37.2149
Mean Absolute Percentage Error: 17.74%
Symmetric Mean Absolute Percentage Error: 21.64%
Root Mean Square Error: 44.2813

Model: .\model-best-retest-shorter.pth, Aggregate Score: 120.5657
Metrics for prediction 1:
Mean Absolute Error: 2.1865
Mean Absolute Percentage Error: 50.21%
Symmetric Mean Absolute Percentage Error: 67.33%
Root Mean Square Error: 2.2210
Metrics for prediction 2:
Mean Absolute Error: 268.2839
Mean Absolute Percentage Error: 94.26%
Symmetric Mean Absolute Percentage Error: 179.26%
Root Mean Square Error: 300.7775

Model: .\current_best_model_b4_lr0.001_e500.pth, Aggregate Score: 124.5978
Metrics for prediction 1:
Mean Absolute Error: 2.7696
Mean Absolute Percentage Error: 63.84

In [8]:
import torchvision.transforms as transforms
import cv2

model = CNN()
model.load_state_dict(torch.load("cnn_model.pth"))
model.eval()

# using cv2 is gross but i like the transforms more
image_path = "test\epsnx0.10_alfax-0.00_betax282.89_epsny0.10_alfay-0.55_betay170.00_epsnz5.00_alfaz0.10_betaz10.00.png"
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = transforms.ToTensor()(image)
image = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])(image)
image = image.unsqueeze(0)

with torch.no_grad():
    output = model(image)
    predicted_variables = output.squeeze().tolist()

print("Predicted variables:", predicted_variables)

FileNotFoundError: [Errno 2] No such file or directory: 'cnn_model.pth'